Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Do the [Plotly Dash](https://dash.plot.ly/) Tutorial, Parts 1 & 2.
- [ ] Add your own stretch goal(s) !

In [0]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module1')

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv('../data/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [4]:
#Number of rows after removing the outliers is
df.shape
#down by 535 rows

(48817, 34)

In [5]:
#checking for Nulls
df.isnull().sum()

bathrooms                  0
bedrooms                   0
created                    0
description             1425
display_address          133
latitude                   0
longitude                  0
price                      0
street_address            10
interest_level             0
elevator                   0
cats_allowed               0
hardwood_floors            0
dogs_allowed               0
doorman                    0
dishwasher                 0
no_fee                     0
laundry_in_building        0
fitness_center             0
pre-war                    0
laundry_in_unit            0
roof_deck                  0
outdoor_space              0
dining_room                0
high_speed_internet        0
balcony                    0
swimming_pool              0
new_construction           0
terrace                    0
exclusive                  0
loft                       0
garden_patio               0
wheelchair_access          0
common_outdoor_space       0
dtype: int64

In [0]:
#Adding the following new featrues
df['bath_bed']=df['bathrooms']+df['bedrooms'] #Number of bathrooms plus bedrooms

In [207]:
perks_columns

['elevator',
 'cats_allowed',
 'hardwood_floors',
 'dogs_allowed',
 'doorman',
 'dishwasher',
 'no_fee',
 'laundry_in_building',
 'fitness_center',
 'pre-war',
 'laundry_in_unit',
 'roof_deck',
 'outdoor_space',
 'dining_room',
 'high_speed_internet',
 'balcony',
 'swimming_pool',
 'new_construction',
 'terrace',
 'exclusive',
 'loft',
 'garden_patio',
 'wheelchair_access',
 'common_outdoor_space']

In [218]:
df['street_address'].value_counts()

3333 Broadway                        174
505 West 37th Street                 167
200 Water Street                     160
90 Washington Street                 142
100 Maiden Lane                      131
401 East 34th Street                 129
2 Gold Street                        120
1 West Street                        119
100 John Street                      115
95 Wall Street                       106
340 East 29th Street                  98
50 West 34th Street                   94
95 Christopher Street                 93
8 Spruce Street                       88
145 East 16th Street                  87
888 Eighth Avenue                     85
550 West 54th Street                  84
10 Hanover Square                     83
166 Second Avenue                     82
505 W 37 St.                          80
1 West St.                            79
561 10 Ave.                           76
236 East 36th Street                  75
140 East 46th Street                  75
1 Astor Place   

In [209]:
df['display_address'].value_counts()

Broadway                                          435
East 34th Street                                  355
Second Avenue                                     349
Wall Street                                       330
West 37th Street                                  287
West Street                                       257
First Avenue                                      244
Gold Street                                       241
York Avenue                                       228
Washington Street                                 227
John Street                                       214
Water Street                                      214
East 39th Street                                  198
East 89th Street                                  195
West 54th Street                                  193
Lexington Avenue                                  189
West 42nd Street                                  184
Christopher Street                                180
Columbus Avenue             

In [204]:
df[['display_address','street_address']]

,display_address,street_address
0,Metropolitan Avenue,792 Metropolitan Avenue
1,Columbus Avenue,808 Columbus Avenue
2,W 13 Street,241 W 13 Street
3,East 49th Street,333 East 49th Street
4,West 143rd Street,500 West 143rd Street
5,West 18th Street,350 West 18th Street
6,West 107th Street,210 West 107th Street
7,West 21st Street,155 West 21st Street
8,Hamilton Terrace,63 Hamilton Terrace
9,522 E 11th,522 E 11th


In [8]:
#Converting interest level into integer or rather cardinal
df['interest_level'].value_counts()

low       33946
medium    11181
high       3690
Name: interest_level, dtype: int64

In [0]:
df['interest_level'] = df['interest_level'].replace({'low':1,'medium':2,'high':3}).astype('int')

In [10]:
df['interest_level'].value_counts()

1    33946
2    11181
3     3690
Name: interest_level, dtype: int64

In [0]:
#Chaging created feature datetime
df['created'] = pd.to_datetime(df['created'],infer_datetime_format=True)

In [15]:
#Getting the boolean columns and adding them to perk columns
perks_columns = []
for col in df:
  if(df[col].nunique()==2):
    perks_columns.append(col)
print(len(perks_columns))
perks_columns

24


['elevator',
 'cats_allowed',
 'hardwood_floors',
 'dogs_allowed',
 'doorman',
 'dishwasher',
 'no_fee',
 'laundry_in_building',
 'fitness_center',
 'pre-war',
 'laundry_in_unit',
 'roof_deck',
 'outdoor_space',
 'dining_room',
 'high_speed_internet',
 'balcony',
 'swimming_pool',
 'new_construction',
 'terrace',
 'exclusive',
 'loft',
 'garden_patio',
 'wheelchair_access',
 'common_outdoor_space']

In [0]:
df['openspaces']=df['roof_deck']+df['outdoor_space']+df['balcony']+df['terrace']+df['loft']+df['garden_patio']

In [0]:
#Getting the feature of number of perks for a place
df['no_of_perks']=df[perks_columns].sum(axis=1)

In [0]:
#Getting the bed to bath ratio
df['bed_per_bathrooms']=df['bedrooms']/df['bathrooms']
#Removing this columns as some values is inf and I currently do not know how to address the inf values


In [150]:
df['bed_per_bathrooms'].isna().sum()

0

In [149]:
df['bed_per_bathrooms'].describe()

count    48817.000000
mean        32.600846
std        558.836894
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max       9999.000000
Name: bed_per_bathrooms, dtype: float64

In [0]:
df['bed_per_bathrooms'] = df['bed_per_bathrooms'].fillna(0)

In [0]:
df['bed_per_bathrooms'] = df['bed_per_bathrooms'].replace(np.inf,9999)

In [26]:
#Now splitting the date into train and test based on created date
# April and May are Train data
#June is the test data
df['created'].describe()

count                   48817
unique                  48148
top       2016-05-14 01:11:03
freq                        3
first     2016-04-01 22:12:41
last      2016-06-29 21:41:47
Name: created, dtype: object

In [30]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,bath_bed,no_of_perks,bed_per_bathrooms
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0,2.0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,1,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,5,2.0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,3,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,3,1.0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2,1.0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,1,4.0


In [151]:
#features are the columns which are either float or int
features = [col for col in df if ((df[col].dtype=='float64') |(df[col].dtype=='int64'))]
print(len(features))
features
#Removing price from the features as it is the target
features.remove('price')
features

33


['bathrooms',
 'bedrooms',
 'latitude',
 'longitude',
 'interest_level',
 'elevator',
 'cats_allowed',
 'hardwood_floors',
 'dogs_allowed',
 'doorman',
 'dishwasher',
 'no_fee',
 'laundry_in_building',
 'fitness_center',
 'pre-war',
 'laundry_in_unit',
 'roof_deck',
 'outdoor_space',
 'dining_room',
 'high_speed_internet',
 'balcony',
 'swimming_pool',
 'new_construction',
 'terrace',
 'exclusive',
 'loft',
 'garden_patio',
 'wheelchair_access',
 'common_outdoor_space',
 'bath_bed',
 'openspaces',
 'bed_per_bathrooms']

In [152]:
train = df[df['created']<='2016-05-31']
test= df[df['created']>'2016-05-31']
train.shape,test.shape

((31839, 37), (16978, 37))

In [174]:
features

['bathrooms',
 'bedrooms',
 'latitude',
 'longitude',
 'interest_level',
 'elevator',
 'cats_allowed',
 'hardwood_floors',
 'dogs_allowed',
 'doorman',
 'dishwasher',
 'no_fee',
 'laundry_in_building',
 'fitness_center',
 'pre-war',
 'laundry_in_unit',
 'roof_deck',
 'outdoor_space',
 'dining_room',
 'high_speed_internet',
 'balcony',
 'swimming_pool',
 'new_construction',
 'terrace',
 'exclusive',
 'loft',
 'garden_patio',
 'wheelchair_access',
 'common_outdoor_space',
 'bath_bed',
 'openspaces',
 'bed_per_bathrooms',
 'no_of_perks']

In [0]:
features = ['latitude', 'longitude','interest_level', 'elevator', 'cats_allowed',
 'hardwood_floors', 'dogs_allowed', 'doorman', 'dishwasher', 'no_fee', 'laundry_in_building',
 'fitness_center', 'pre-war', 'laundry_in_unit', 'dining_room', 'high_speed_internet',
 'swimming_pool', 'new_construction', 'exclusive', 'wheelchair_access', 'common_outdoor_space',
 'bath_bed', 'openspaces', 'bed_per_bathrooms', 'no_of_perks']

In [0]:
target = np.log(train['price'])

In [177]:
#Now training the model on train data on Linear Regression model
from sklearn.linear_model import LinearRegression
model = LinearRegression()
#Fitting the data on train data
model.fit(train[features],target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [178]:
coefficients.sort_values(ascending=False)

openspaces              1.632372e+12
bath_bed                4.111244e+11
doorman                 4.361554e+10
laundry_in_unit         4.361554e+10
elevator                4.361554e+10
dishwasher              4.361554e+10
wheelchair_access       4.361554e+10
fitness_center          4.361554e+10
dining_room             4.361554e+10
dogs_allowed            4.361554e+10
exclusive               4.361554e+10
swimming_pool           4.361554e+10
cats_allowed            4.361554e+10
no_fee                  4.361554e+10
laundry_in_building     4.361554e+10
pre-war                 4.361554e+10
hardwood_floors         4.361554e+10
new_construction        4.361554e+10
common_outdoor_space    4.361554e+10
high_speed_internet     4.361554e+10
latitude                1.961060e-01
bed_per_bathrooms       3.661015e-05
interest_level         -1.230621e-01
longitude              -4.133463e+00
no_of_perks            -4.361554e+10
bathrooms              -4.111244e+11
bedrooms               -4.111244e+11
t

In [179]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
#Based on model predicting the price for both train  and test data
y_train = np.exp(model.predict(train[features]))
y_test = np.exp(model.predict(test[features]))
train_price = train['price']
test_price = test['price']
rmse_train = np.sqrt(mean_squared_error(train_price,y_train))
rmse_test = np.sqrt(mean_squared_error(test_price,y_test))
#MAE for both train and test data
print(f'The MAE for Train :{mean_absolute_error(train_price,y_train):.2f} and Test:{mean_absolute_error(test_price,y_test):.2f}')
print(f'The R2 score for Train :{r2_score(train_price,y_train):.2f} and Test:{r2_score(test_price,y_test):.2f}')
print(f'The RMSE score for Train :{rmse_train:.2f} and Test:{rmse_test:.2f}')


The MAE for Train :644.92 and Test:649.09
The R2 score for Train :0.62 and Test:0.63
The RMSE score for Train :1088.31 and Test:1071.46


In [203]:
#Organising the code
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
# Read New York City apartment rental listing data
df = pd.read_csv('../data/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]
df['bath_bed']=df['bathrooms']+df['bedrooms'] #Number of bathrooms plus bedrooms
df['bathandbed']=df['bathrooms']*df['bedrooms']
df['cats_dogs']=df['cats_allowed']+df['dogs_allowed']
#Converting interest level into integer or rather cardinal
df['interest_level'] = df['interest_level'].replace({'low':1,'medium':2,'high':3}).astype('int')
df['openspaces']=df['roof_deck']+df['outdoor_space']+df['balcony']+df['terrace']+df['loft']+df['garden_patio']
#Getting the bed to bath ratio
df['bed_per_bathrooms']=df['bedrooms']/df['bathrooms']
df['bed_per_bathrooms'] = df['bed_per_bathrooms'].fillna(0)
df['bed_per_bathrooms'] = df['bed_per_bathrooms'].replace(np.inf,9999)
#Getting the bath to bed ratio
df['bath_per_bed']=df['bathrooms']/df['bedrooms']
df['bath_per_bed'] = df['bath_per_bed'].fillna(0)
df['bath_per_bed'] = df['bath_per_bed'].replace(np.inf,9999)
#Chaging created feature datetime
df['created'] = pd.to_datetime(df['created'],infer_datetime_format=True)
#Getting the boolean columns and adding them to perk columns
perks_columns = []
for col in df:
  if(df[col].nunique()==2):
    perks_columns.append(col)
#Getting the feature of number of perks for a place
df['no_of_perks']=df[perks_columns].sum(axis=1)
features = [col for col in df if ((df[col].dtype=='float64') |(df[col].dtype=='int64'))]
#Removing price from the features as it is the target
features.remove('price')
#features.remove('latitude')
#features.remove('longitude')
train = df[df['created']<='2016-05-31']
test= df[df['created']>'2016-05-31']
model = LinearRegression(normalize=True)
#Fitting the data on train data
#Taking the log of the price
model.fit(train[features],np.log(train['price']))
#print(f'The intercept of the model is {model.intercept_:,.0f}')
coefficients = pd.Series(model.coef_,features).sort_values(ascending=False)
print(coefficients.to_string())
#Based on model predicting the price for both train  and test data
#Taking a exp of the prediction to bring the price back to scale
y_train = np.exp(model.predict(train[features]))
y_test = np.exp(model.predict(test[features]))
train_price = train['price']
test_price = test['price']
rmse_train = np.sqrt(mean_squared_error(train_price,y_train))
rmse_test = np.sqrt(mean_squared_error(test_price,y_test))
#MAE for both train and test data
print(f'The MAE for Train :{mean_absolute_error(train_price,y_train):.2f} and Test:{mean_absolute_error(test_price,y_test):.2f}')
print(f'The R2 score for Train :{r2_score(train_price,y_train):.2f} and Test:{r2_score(test_price,y_test):.2f}')
print(f'The RMSE score for Train :{rmse_train:.2f} and Test:{rmse_test:.2f}')


openspaces              1.716207e+12
bath_bed                3.714533e+11
dogs_allowed            6.864619e+10
cats_allowed            6.864619e+10
no_of_perks             1.682458e+10
latitude                1.934127e-01
bed_per_bathrooms       3.904966e-05
bath_per_bed           -7.098636e-06
bathandbed             -1.816089e-02
interest_level         -1.208971e-01
longitude              -4.165015e+00
doorman                -1.682458e+10
laundry_in_unit        -1.682458e+10
elevator               -1.682458e+10
dishwasher             -1.682458e+10
wheelchair_access      -1.682458e+10
dining_room            -1.682458e+10
fitness_center         -1.682458e+10
exclusive              -1.682458e+10
swimming_pool          -1.682458e+10
no_fee                 -1.682458e+10
laundry_in_building    -1.682458e+10
pre-war                -1.682458e+10
hardwood_floors        -1.682458e+10
new_construction       -1.682458e+10
common_outdoor_space   -1.682458e+10
high_speed_internet    -1.682458e+10
c

In [45]:
# Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
# Engineer at least two new features. (See below for explanation & ideas.)
# Fit a linear regression model with at least two features.
# Get the model's coefficients and intercept.
# Get regression metrics RMSE, MAE, and  R2 , for both the train and test data.
# What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
# As always, commit your notebook to your fork of the GitHub repo.


bathrooms               0
bedrooms                0
latitude                0
longitude               0
interest_level          0
elevator                0
cats_allowed            0
hardwood_floors         0
dogs_allowed            0
doorman                 0
dishwasher              0
no_fee                  0
laundry_in_building     0
fitness_center          0
pre-war                 0
laundry_in_unit         0
roof_deck               0
outdoor_space           0
dining_room             0
high_speed_internet     0
balcony                 0
swimming_pool           0
new_construction        0
terrace                 0
exclusive               0
loft                    0
garden_patio            0
wheelchair_access       0
common_outdoor_space    0
bath_bed                0
no_of_perks             0
dtype: int64